In [2]:
import os 
import pandas as pd
import numpy as np
import copy
import warnings
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

In [3]:
file = os.path.expanduser('~/Desktop/titanic_train.csv')
df = pd.read_csv(file)

In [4]:
train_y = df['Survived']
df = df.drop(['PassengerId','Survived'], axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df['Sex'] = df['Sex'].fillna('None')
mean_df = df.groupby(['Sex'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Sex'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Sex'])['Fare'].median().reset_index()
max_df = df.groupby(['Sex'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Sex'])
temp = pd.merge(temp, median_df, how='left', on=['Sex'])
temp = pd.merge(temp, max_df, how='left', on=['Sex'])
temp.columns = ['Sex', 'combine_Mean', 'combine_Mode', 'combine_Median', 'combine_Max']
temp

,Sex,combine_Mean,combine_Mode,combine_Median,combine_Max
0,female,44.479818,7.75,23.0,512.3292
1,male,25.523893,8.05,10.5,512.3292


In [6]:
df = pd.merge(df, temp, how='left', on=['Sex'])
df = df.drop(['Sex'],axis=1)
df.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,combine_Mean,combine_Mode,combine_Median,combine_Max
0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,25.523893,8.05,10.5,512.3292
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,44.479818,7.75,23.0,512.3292
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,44.479818,7.75,23.0,512.3292
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,44.479818,7.75,23.0,512.3292
4,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,25.523893,8.05,10.5,512.3292


In [7]:
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'combine_Mean', 'combine_Mode', 'combine_Median', 'combine_Max']



,Pclass,Age,SibSp,Parch,Fare,combine_Mean,combine_Mode,combine_Median,combine_Max
0,3,22.0,1,0,7.2500,25.523893,8.05,10.5,512.3292
1,1,38.0,1,0,71.2833,44.479818,7.75,23.0,512.3292
2,3,26.0,0,0,7.9250,44.479818,7.75,23.0,512.3292
3,1,35.0,1,0,53.1000,44.479818,7.75,23.0,512.3292
4,3,35.0,0,0,8.0500,25.523893,8.05,10.5,512.3292


In [10]:
df_minus = df.drop(['combine_Mean','combine_Mode','combine_Median','combine_Max'],axis=1)
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_y, cv=5).mean()

0.7038635542329971

In [11]:
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_y, cv=5).mean()

0.7822774202448624